主要讲解的是如何使用Redis查询来代替传统关系数据库查询，以及如何使用redis来完成一些关系数据库没办法完成的任务

# 登录和Cookie缓存
* 签名Cookie：所有验证需要的信息都放在Cookie里，并且会有一段签名来验证数据是否改动
* 令牌Cookie：相关的内容存储在后台数据库  
应对负载量大的问题，需要使用Redis重新实现登录cookie功能，取代目前由关系型数据库实现的登录cookie功能

In [1]:
import time
import redis
import json
conn = redis.Redis()

In [2]:
# 创造一个散列用来存储已经登录的客户以及其cookie
# 如果存在，返回用户id，如果不存在，返回None
def check_token(conn, token):
    return conn.hget('login:', token)

In [3]:
# 更新令牌，将某一个token赋予某一个user，将token放入有序集合，如果item非None，也会被放入一个有序集合
def update_token(conn, token, user, item=None):
    timestamp = time.time()
    conn.hset('login:', token, user)
    conn.zadd('recent:', token, timestamp)
    if item:
        # 通过token来记录浏览过的商品
        conn.zadd('viewed:' + token, item, timestamp)
        # 有序集合是从小到大排列的，所以从0开始删除到倒数第26位，就剩下了25个
        conn.zremrangebyrank('viewed:' + token, 0, -26)

因为存储会话数据会随着时间的推移而不断的增加，因此我们需要定期清理旧的会话数据  
清理旧的会话程序由一个循环构成，会检查每个存储令牌有序集合的大小，如果超过阈值，会删除最旧的100个  
如果未超过限制，程序会先休眠1秒，然后再重新检查

In [4]:
QUIT = False
LIMIT = 100000
def clean_session(conn):
    while not QUIT:
        size = conn.zcard('recent:')
        if size <= LIMIT:
            time.sleep(1)
            continue
        
        end_index = min(size - LIMIT, 100)
        # 从低到高排序的
        tokens = conn.zrange('recent:', 0, end_index - 1)
        
        session_keys = []
        for token in tokens:
            # 所谓的session，就是所有商品浏览记录
            session_keys.append('viewed:' + token)
        
        # 批量删除所有浏览记录
        conn.delete(*session_keys)
        # 批量删除登录记录
        conn.hdel('login:', *tokens)
        # 批量删除总历史记录
        conn.zrem('recent:', *tokens)

类似以上的清理函数，可以用cronjob的形式，也可以用守护进程的形式

# 使用Redis实现购物车

最早是使用cookie实现购物车的，也就是会把购物车内的信息全部放在cookie里，虽然这样的设计能够避免数据库写入，但是如果购物车的内容多了，请求发送和处理的时间就会增加。  
每个用户的购物车都是一个散列，存储了商品ID和商品订购数量之间的映射

In [5]:
# 往购物车中加入物品
def add_to_cart(conn, session, item, count):
    if count <= 0:
        conn.hrem('cart:' + session, item)
    else:
        conn.hset('cart:' + session, item, count)

因为有了购物车的概念，所以在会话清理的时候，将就会话对应的购物车一并删除也是有必要的

In [6]:
QUIT = False
LIMIT = 100000
def clean_full_session(conn):
    while not QUIT:
        size = conn.zcard('recent:')
        if size <= LIMIT:
            time.sleep(1)
            continue
        
        end_index = min(size - LIMIT, 100)
        # 从低到高排序的
        sessions = conn.zrange('recent:', 0, end_index - 1)
        
        session_keys = []
        for sess in sessions:
            # 所谓的session，就是所有商品浏览记录
            session_keys.append('viewed:' + sess)
            session_keys.append('cart:' + sess)
        
        # 批量删除所有浏览记录
        conn.delete(*session_keys)
        # 批量删除登录记录
        conn.hdel('login:', *tokens)
        # 批量删除总历史记录
        conn.zrem('recent:', *tokens)

# 网页缓存

如果缓存页面不存在，函数会生成页面并且将其缓存在redis中5分钟

In [7]:
def cache_request(conn, request, callback):
    if not can_cache(conn, request):
        return callback(request)
    
    page_key = 'cache:' + hash_request(request)
    content = conn.get(page_key)
    if not content:
        content = callback(request)
        # 直接限制好了expire的时间
        conn.setex(page_key, content, 300)
    return content

# 数据行缓存

用来应付大量请求同时访问同一数据的情况  
需要两个有序集合，key都是数据行id，一个用来记录延迟时间，一个用来记录调度时间，延迟时间记录应该可以仅仅是个散列   
在处理一条数据的时候，守护进程先看是否到了调度时间，如果到了，得到数据，并且根据延迟时间将调度任务放入调度集合

In [8]:
# 调度数据行缓存
# 仅仅是一个启动器，后续的循环调度有守护进程自己实现
def schedule_row_cache(conn, row_id, delay):
    conn.zadd('delay:', row_id, delay)
    conn.zadd('schedule', row_id, time.time())

In [9]:
# 守护进程
def cache_rows(conn):
    while not QUIT:
        next = conn.zrange('schedule:', 0, 0, withscores=True)
        now = time.time()
        if not next or next[0][1] > now:
            time.sleep(0.05)
            continue
            
        row_id = next[0][0]
        
        # 获得延迟值
        delay = conn.zscore('delay:', row_id)
        if delay <= 0:
            conn.zrem('delay:', row_id)
            conn.zrem('schedule:', row_id)
            conn.delete('inv:' + row_id)
        
        # 用来表示从数据库中获得相关数据
        row = Inventory.get(row_id)
        conn.zadd('schedule:', row_id, now + delay)
        conn.set('inv:' + row_id, json.dumps(row.to_dict()))

由此可以不断的更新数据行的值，保证其一定存在，但就一定要做一套守护进程，由守护进程对这些数据进行维护

# 网页分析
并不是所有网页都需要缓存，可以选择最流行的页面进行缓存  
要实现最受欢迎的页面/商品的排行，只要在 update_token 页面加一个计数器就可以了

In [10]:
def update_token(conn, token, user, item=None):
    timestamp = time.time()
    conn.hset('login:', token, user)
    conn.zadd('recent:', token, timestamp)
    if item:
        # 通过token来记录浏览过的商品
        conn.zadd('viewed:' + token, item, timestamp)
        # 有序集合是从小到大排列的，所以从0开始删除到倒数第26位，就剩下了25个
        conn.zremrangebyrank('viewed:' + token, 0, -26)
        # 因为有序集合是从小到大排序，所以最流行的数值应该越小
        conn.zincrby('viewed:', itemm, -1)

需要定期修剪有序集合的长度并调整已有元素的分值（不然历史的商品有可能会长期霸占）  
调整分值的动作可以通过zinterstore的方式得到  
同样需要守护进程 

In [11]:
def rescale_viewed(conn):
    while not QUIT:
        conn.zremrangebyrank('viewed:', 0, -20001)
        # 得到的数字传给原值，同时 {} 的用法代表了对这个集合的所有数值赋予一个权重
        conn.zinterstore('viewed:', {'viewed:', 0.5})
        time.sleep(300)

In [12]:
# 重写can_cache,来确定是否需要被缓存
def can_cache(conn, request):
    item_id = extract_item_id(request)
    if not item_id or is_dynamic(request):
        return False
    rank = conn.zrank('viewed:', item_id)
    return rank is not None and rank < 10000